## Chcem vytvorit AI travel AGENTA from Scratch
#### Vdaka tomu pochopim lepsie AI agentoch a ich "ins and outs"
#### Vytvorim AI agenta ktory bude vykonavat ulohy v tomto poradi: Think,Act,Learn
#### Zacnem od zakladov bez LANGGRAPHU
#### ReAct PATTERN - Vykonava sa cyklus, az kym je uloha splnena.. Ako prve prebieha Thought proces , kde vdaka LM/Chat GPT, analyzuje a uvazuje/mysli (reasoning). Vdaka analyze sa rozhodne o dalsich krokoch(moze nieco vypocitat, API call, spytat sa pouzivatela etc.). Agent receivne observation, vysledok/outcome danej vykonavanej akcie. Cyklus "Thought,Action a Observation" sa opakuje

---


### Ako prve si potrebujem pripravit prostredie


#### Nainstalujem si potrebne kniznice a vytvorim si API keys pre openAI a weather app

In [5]:
%pip install -q openai sentencepiece transformers


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(dotenv_path="/Users/katka/AI AGENT/enviroments.env",override=True)
os.environ.get("OPENAI_API_KEY")
#os.environ.get("WEATHER")
print('API key loaded',os.environ.get("OPENAI_API_KEY") is not None)
#print('API key loaded',os.environ.get("WEATHER") is not None)

API key loaded True


## Teraz pridam potrebne moduly

In [2]:
import openai
import re
import os
import requests
from datetime import datetime,timedelta

Otestujem konfiguraciu OPEN AI 

In [5]:
from openai import OpenAI
client=OpenAI()

completion=client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            'role':'user',
            'content':"Write something short but funny"
        }
    ]
)

completion.choices[0].message.content

"Why don't skeletons fight each other? They don't have the guts!"

Nasledne potrebujem vytvorit triedu pre Travel Agenta , ktory bude vykonavat dane akcie a prepocitavat napriklad budget

In [97]:
class TravelAgent:
    def __init__(self,prompt,budget=1000):  # tu len pridam jednotlive self premenne
        self.system_prompt=prompt
        self.budget=budget
        self.messages=[
            {'role':'system','content': prompt},
            {'role':'system','content': f'The curent budget is ${self.budget}'}
        ]
        self.memory=[]
    
    def __call__(self, user_input):
        self.messages.append({'role':"user",'content': user_input})
        response=self.execute()
        self.messages.append({'role':'user','content': response})
        return response
    
    def execute(self):
        completion=client.chat.completions.create(
            model='gpt-4o-mini',
            messages=self.messages,
            temperature=0.7,
        )
        return completion.choices[0].message.content
    
    def add_memory(self,action,result):
        self.memory.append(f'Action: { action},Result: { result}')
        

    def set_budget(self,amount):
        self.budget=amount
        self.messages.append({'role':'system','content':f"The current budget is: ${self.budget}."})

    
    def get_budget(self):
        return self.budget



## Teraz vytvorime nejake TOOLS

In [11]:
def currency(amount,from_currency,to_currency):
    url=f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
    response=requests.get(url)
    data=response.json()
    rate=data['rates'][to_currency]
    converted_amount=amount*rate
    return round(converted_amount,2)


In [18]:
currency(100,"CZK","EUR")

3.98

## Weather Tool

In [ ]:


def weather(location:str,days_ahead:int=0)-> str:
    load_dotenv(dotenv_path="/Users/katka/AI AGENT/enviroments.env",override=True)
    api_key=os.environ.get("OPENWEATHERMAP_API_KEY")
    
    if not api_key:
        raise Exception("Variable not set.")

    if days_ahead > 0:
        target_date = (datetime.now()+ timedelta(days=days_ahead)).strftime('%Y-%m-%d')
    else:
        target_date=None

    base_url="http://api.openweathermap.org/data/2.5/weather"

    params={
        "q":location,
        "appid":api_key,
        "units":"metric",
    }
    response=requests.get(base_url,params=params)
    if response.status_code==200:
        data=response.json()
        print(data)
        if target_date:
            return f"THe weather in {data['name']},on {target_date} is currently {data['weather'][0]['description']}"
        else:
            return f"The curent weather in {data['name']}, is currently {data['weather'][0]['description']}"
    else:
        return f"Error, {response.status_code}-{response.reason}"



    


In [70]:
weather('Bratislava',3)

{'coord': {'lon': 17.1067, 'lat': 48.1482}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': -0.56, 'feels_like': -0.56, 'temp_min': -1.19, 'temp_max': -0.16, 'pressure': 1032, 'humidity': 89, 'sea_level': 1032, 'grnd_level': 1009}, 'visibility': 10000, 'wind': {'speed': 1.03, 'deg': 0}, 'clouds': {'all': 0}, 'dt': 1735325114, 'sys': {'type': 2, 'id': 2006545, 'country': 'SK', 'sunrise': 1735281692, 'sunset': 1735311826}, 'timezone': 3600, 'id': 3060972, 'name': 'Bratislava', 'cod': 200}


'THe weather in Bratislava,on 2024-12-30 is currently clear sky'

### Translation Tool

In [103]:
from transformers import pipeline


def translation(text,target_language):
    try:
        translator = pipeline("translation",model="facebook/m2m100_418M")
        translation = translator(text,src_lang="en",tgt_lang=target_language)[0]['translation_text']
        return translation
    except Exception as e:
        print(f"Error:{e}")
        return "Translation failed."



In [108]:
from transformers import pipeline

def translation(text, target_language):
    try:
        # Initialize the translation pipeline with the model
        translator = pipeline("translation", model="facebook/m2m100_418M", device=-1)  # Use CPU

        # Ensure both `src_lang` and `tgt_lang` are set correctly
        translated = translator(
            text,
            src_lang='en',
            tgt_lang=target_language
        )
        return translated[0]['translation_text']
    except Exception as e:
        print(f"Error: {e}")
        return "Translation failed."


In [109]:
text="Dad, i love you."
language='fr'
translation(text,language)

Device set to use cpu


'Papa, je t’aime'

### Prompt

In [ ]:
travel_agent_prompt="""You are Travel Agent Assistant mostly for girls and Women.
 Your role is to help others to plan trips by providing information about destination,weather ,currency and language translation etc.
 
 You run in a loop of Thought, Action, PAUSE and Observation.
 Use Thought for analyzing and reasoning user's request.
 Use Action to perform one of the available actions and then return PAUSE.
 Use Observation as result of running those actions.

 Your available actions are:

 1. currency:
 e.g., currency: 100,USD ,EUR
 Converts given amount from one currency to other.

 2. weather:
 e.g., weather: Bratislava , 3
 Gets the forecast for specified city and number of days ahead.

 3. translation:
 e.g., translation: Hello daddy, how are you?,cz
 Translates given text to the specified target language (cz for Chzech language)

 But always consider users budget when making suggestions.


 Example session:

 User: Iam planning trip to Zurich next week. What should I know if i have 1000 euros budget ?

 Thought: I should check the weather in Zurich for next week and provide some basic phrases.
 Action: weather: Zurich , 7
 PAUSE

 Observation: Zurich,cloudy, 20°C

 Thought: Now if i have the weather information , i should provide a simple phrase that are use in SWITZERLAND ..and its translation.
 Action: translation: Hello. How are you ?, DE
 PAUSE 

 Observation: Hallo, wie geht es dir?

 Thought: Now i need to check for currency..
 Action: currency: 1000,EUR,CHF
 PAUSE

 Observation: 937.09 CHF

 
 """

### Dostupne tools

In [116]:
available_tools={
    'currency': currency,
    'weather': weather,
    'translation': translation,
}


## THE MAIN CHARACTER HERE 
 #### najdolezitejsia methoda

In [117]:
def plan_trip(question,max_iter=5):
    agent=TravelAgent(travel_agent_prompt)

    # budget 
    budget_match=re.match(r'\$(\d+)',question)
    if budget_match:
        budget = int(budget_match.group(1))
        agent.set_budget(budget)
    
    next_prompt= question

    action_re = re.compile(r'^Action: (\w+): (.+)$')

    for i in range(max_iter):
        result=agent(next_prompt)
        print(result)

        actions=[action_re.match(a) for a in result.split('\n') if action_re.match(a)]

        if actions:
            action,action_input=actions[0].groups()

            if action not in available_tools:
                print("Unknown action:{action}")
                continue

            print(f"running {action}:{action_input} ")

            try:
                if action =="currency":
                    amount,from_currency,to_currency=action_input.split(',')
                    observation=available_tools[action](float(amount),from_currency.strip(),to_currency.strip())


                elif action=="weather":
                    location,*days=action_input.split(',')
                    days_ahead=int(days[0]) if days else 0
                    observation=available_tools[action](location.strip())

                elif action=="translation":
                    if ',' in action_input:
                        text,target_lang=action_input.rsplit(',',1)
                    else:
                        text,target_lang=action_input.rsplit(None,1)
                    observation=available_tools[action](text.strip(),target_lang.strip())
                
                print(f"Observation: {observation}")
                agent.add_memory(action,observation)
                next_prompt=f'Observation: {observation}'
            except Exception as e:
                print(f"Error: {e}")
                next_prompt=f'Error,unable to do this action:{action}.Please try again'
        else:
            return 

In [ ]:
question="Hi, can you tell me what should i do , if I'am planning , going to London and my budget is 2000 euro?"
plan_trip(question)

Thought: I should check the weather in London for next week and provide some basic phrases useful for communication in the UK. Also, I need to consider the currency conversion since the budget is in euros. 

Action: weather: London, 7 
PAUSE
running weather:London, 7  
{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 5.21, 'feels_like': 2.69, 'temp_min': 4.45, 'temp_max': 6.06, 'pressure': 1029, 'humidity': 95, 'sea_level': 1029, 'grnd_level': 1024}, 'visibility': 10000, 'wind': {'speed': 3.09, 'deg': 290}, 'clouds': {'all': 100}, 'dt': 1735412618, 'sys': {'type': 2, 'id': 2075535, 'country': 'GB', 'sunrise': 1735373168, 'sunset': 1735401482}, 'timezone': 0, 'id': 2643743, 'name': 'London', 'cod': 200}
Observation: The curent weather in London, is currently overcast clouds
Thought: Now that I have the weather information, I should provide a simple phrase that is 

Device set to use cpu


Error: 'EN'
Observation: Translation failed.
Thought: Since the translation failed, I will try a different phrase that is commonly used in the UK and translate it to English. 

Action: translation: Good morning! How are you?, EN 
PAUSE
running translation:Good morning! How are you?, EN  


Device set to use cpu


Error: 'EN'
Observation: Translation failed.
Thought: It seems that I'm having trouble with the translation. Instead, I will provide a common phrase used in the UK without translation, and focus on other useful information such as currency conversion.

Action: currency: 2000,EUR,GBP 
PAUSE
running currency:2000,EUR,GBP  
Observation: 1660.0
Thought: I now have the currency conversion result. The budget of 2000 euros is approximately 1660 British pounds. I should summarize the information gathered so far for your trip to London, including the weather and a useful phrase.

Action: Provide summary information. 
PAUSE
